In [1]:
#load
print('loading data...')
import numpy as np
file=open('adult.data')
r_X=[]
r_y=[]
for line in file.readlines():
    items=str(line).split(',')
    r_X.append(items[:-1])
    r_y.append(items[-1]==' >50K\n')
del r_X[32561],r_y[32561]
y=np.array(r_y).astype('int')
X=np.array(r_X)
def count(row):
    index=dict()
    names=set()
    for i in row:
        if i not in names:
            names.add(i)
    for i in names:
        i_num=(row[y==1]==i).sum()
        index[i]=i_num
    for i in range(len(index.keys())):
        max_index=min(index)
        row[row==max_index]=index[max_index]
        del index[max_index]
    return row.astype('float')
print(X[0])
for num in [1,3,5,6,7,8,9,13]:
    count(X[:,num])
X=np.matrix(X.astype(float)).T
X=X/(X.max(axis=1)-X.min(axis=1))
print(X[9,:])
print('done!')

loading data...
['39' ' State-gov' ' 77516' ' Bachelors' ' 13' ' Never-married'
 ' Adm-clerical' ' Not-in-family' ' White' ' Male' ' 2174' ' 0' ' 40'
 ' United-States']
[[1.21502827 1.21502827 1.21502827 ... 0.21502827 1.21502827 0.21502827]]
done!


In [2]:
X.shape,y.shape,(y==0).sum()/X.shape[1],X

((14, 32561),
 (32561,),
 0.7591904425539756,
 matrix([[0.53424658, 0.68493151, 0.52054795, ..., 0.79452055, 0.30136986,
          0.71232877],
         [0.07112633, 0.14587951, 1.        , ..., 1.        , 1.        ,
          0.12532742],
         [0.05264531, 0.056581  , 0.14645685, ..., 0.10317029, 0.13684275,
          0.19554679],
         ...,
         [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
          0.        ],
         [0.40816327, 0.13265306, 0.40816327, ..., 0.40816327, 0.20408163,
          0.40816327],
         [1.        , 1.        , 1.        , ..., 1.        , 1.        ,
          1.        ]]))

In [12]:
X_train,y_train,X_val,y_val=X[:,:300],y[:300],X[:,3030:3035],y[3030:3035]
X_train.shape

(14, 300)

In [13]:
class node(object):
    def __init__(self, dataset, depth=0, parent=None, median=None):
        self.parent = parent
        self.median = median
        self.depth = depth
        self.data = dataset
        self.create_child_node()

    def create_child_node(self):
        if self.data.shape[1] <= 1:
            if self.data.shape[1] == 1:
                self.value = self.data
            else:
                self.value = None
            return
        else:
            self.median = np.median(self.data[(self.depth)], axis=1)
            median = self.median
            left = self.data[np.repeat(self.data[self.depth, :] < median, self.data.shape[0]).reshape(
                self.data.shape[1], self.data.shape[0]).T]
            left = left.reshape(self.data.shape[0], int(
                left.shape[1] / self.data.shape[0]))
            right = self.data[np.repeat(self.data[self.depth, :] >= median, self.data.shape[0]).reshape(
                self.data.shape[1], self.data.shape[0]).T]
            right = right.reshape(self.data.shape[0], int(
                right.shape[1] / self.data.shape[0]))
            self.left = node(depth=(self.depth + 1) %
                             self.data.shape[0], parent=self, dataset=left)
            self.right = node(depth=(self.depth + 1) %
                              self.data.shape[0], parent=self, dataset=right)
            self.value = self.data[:, 0]


class kdd_KNN(object):
    def fit(self, X, y):
        self.X=X
        self.kdtree = node(self.X)

    def predict(self, x):
        x = x.T
        result = []
        i = 0
        for xi in x:
            i += 1
            print(i)
            xi = xi.T
            cloest = self.find(xi, self.kdtree)
            result.append(
                y[np.where((self.X == cloest).sum(axis=0) == xi.shape[0])[1]][0])
        return result

    def find(self, x, kdtree,depth=0):
        root = kdtree
        locate = kdtree
        cloest = locate.value
        side = None
        while locate.data.shape[1] > 1:
            if x[depth % x.shape[0]] < locate.median:
                locate = locate.left
                side = 'l'
            else:
                locate = locate.right
                side = 'r'
            depth+=1
            while locate.data.shape[1] == 0:
                locate = locate.parent
                if side == 'r':
                    locate = locate.left
                else:
                    locate = locate.right
            if (np.array(locate.value - x)**2).sum() < (np.array(cloest - x)**2).sum() and locate.value.shape[1] != 0:
                cloest = locate.value
        cloest = locate.data
        while(locate != root):
            locate = locate.parent
            depth-=1
            if side == 'r':
                other = self.find(x, locate.left,depth+1)
            else:
                other = self.find(x, locate.right,depth+1)

            if (np.array(other - x)**2).sum() < (np.array(cloest - x)**2).sum() and other.shape[1] != 0:
                cloest = other
        return cloest

In [14]:
clf=kdd_KNN()
clf.fit(X_train,y)
pre_y=clf.predict(X_val)

1
2
3
4
5


In [11]:
pre_y,y_val

([0, 0, 0, 1, 0], array([0, 0, 1, 0, 0]))